Assumed donor progression of engagement:

1. one time, qgiv event participant, VT
2. recurring/pledge
3. donor login/givi account
4. P2P fundraiser

In [2]:
import sys, datetime
import pandas as pd
import numpy as np

sys.path.insert(1, '../../scripts/')
from s3_support import *

In [3]:
df = get_dataframe_from_file("qgiv-stats-data", "donor.agg.csv")

In [4]:
print(len(df))
df.head(3)

1000000


,avg_amount,don_form_amount,don_form_count,donations_amt,donations_count,donor,events_amt,events_tickets,fb_amount,fb_count,...,registrations_amt,registrations_count,sms_amount,sms_count,smspledge_amt,smspledge_count,total_amount,transactions,vt_amount,vt_count
0,540.0,540.0,1,540.0,1,844e51be16713addd41c11721d969121762a4129,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0.0,540.0,1,0.0,0
1,50.0,0.0,0,50.0,1,6dd2c0381f68bb29be88f93dc214351028f12761,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0.0,50.0,1,0.0,0
2,42.5,85.0,2,100.0,2,1263c3810eec2ddf9d4ee4ef273a33ddba90bd52,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0.0,85.0,2,0.0,0


In [70]:
print("To best understand our underlying data, let's get an idea of the time ranges available")

df['first_transaction'] = pd.to_datetime(df['first_transaction'])
df.groupby(df['first_transaction'].dt.year)['first_transaction'].count()

To best understand our underlying data, let's get an idea of the time ranges available


first_transaction
2017     69449
2018     67849
2019     96529
2020    766173
Name: first_transaction, dtype: int64

In [5]:
df.describe()

,avg_amount,don_form_amount,don_form_count,donations_amt,donations_count,events_amt,events_tickets,fb_amount,fb_count,givi_amount,...,registrations_amt,registrations_count,sms_amount,sms_count,smspledge_amt,smspledge_count,total_amount,transactions,vt_amount,vt_count
count,1.000000e+06,1.000000e+06,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.0,1000000.0,1.000000e+06,1000000.000000,1.000000e+06,1000000.0
mean,2.387159e+02,5.063131e+02,2.415276,6.029847e+02,3.277204,27.187518,0.601244,0.624786,0.011402,2.558739,...,1.910249,0.158346,3.613920,0.046677,0.0,0.0,6.519615e+02,3.802609,4.537194e+01,0.0
std,1.000064e+05,2.000114e+05,16.147866,2.027810e+05,287.329919,587.966250,59.761076,488.268689,9.954028,166.406984,...,25.733966,0.750509,641.017055,12.919308,0.0,0.0,2.029312e+05,305.314077,2.738678e+04,0.0
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,1.000000,0.000000e+00,0.0
25%,2.500000e+01,0.000000e+00,0.000000,2.000000e+01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.604000e+01,1.000000,0.000000e+00,0.0
50%,5.000000e+01,2.500000e+01,1.000000,5.000000e+01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,8.420000e+01,1.000000,0.000000e+00,0.0
75%,1.029000e+02,1.432925e+02,1.000000,2.000000e+02,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.280000e+02,2.000000,0.000000e+00,0.0
max,1.000000e+08,2.000000e+08,12837.000000,2.000000e+08,286885.000000,506612.040000,59383.000000,487648.200000,9952.000000,101896.770000,...,10000.000000,117.000000,632479.490000,12897.000000,0.0,0.0,2.000000e+08,304838.000000,2.738428e+07,0.0


In [6]:
count_cols = [c for c in df.columns if '_count' in c or 'rec' in c]
skipped = []
for c in count_cols:
    _df = df[df[c]>0]
    if len(_df) == 0:
        skipped.append(c)
        continue
    print("{} ({:,}; {:.0f}%):".format(c, len(_df), (len(_df) / len(df)) * 100.))
    print("\tmean amount: {:,.2f}".format(_df['avg_amount'].mean()))
    print("\tmedian amount: {:,.2f}".format(_df['avg_amount'].median()))
    print("\tmean {} count: {:,.2f}".format(c, _df[c].fillna(0).mean()))
    print("\tmean all count: {:,.2f}".format(_df[count_cols].sum().mean()))
print("\nSkipped: {}".format(", ".join(skipped)))

don_form_count (616,713; 62%):
	mean amount: 330.24
	median amount: 52.12
	mean don_form_count count: 3.92
	mean all count: 36,160,326.10
donations_count (823,406; 82%):
	mean amount: 274.60
	median amount: 51.97
	mean donations_count count: 3.98
	mean all count: 38,032,457.96
fb_count (348; 0%):
	mean amount: 151.14
	median amount: 58.81
	mean fb_count count: 32.76
	mean all count: 531,754.89
givi_count (5,445; 1%):
	mean amount: 136.30
	median amount: 53.33
	mean givi_count count: 3.38
	mean all count: 89,298.85
kiosk_count (6,358; 1%):
	mean amount: 118.94
	median amount: 50.00
	mean kiosk_count count: 20.12
	mean all count: 745,525.69
mobile_count (116,329; 12%):
	mean amount: 121.94
	median amount: 46.67
	mean mobile_count count: 2.77
	mean all count: 2,742,233.75
mobilevt_count (616,713; 62%):
	mean amount: 330.24
	median amount: 52.12
	mean mobilevt_count count: 3.92
	mean all count: 36,160,326.10
p2p_count (286,400; 29%):
	mean amount: 94.79
	median amount: 50.00
	mean p2p_coun

# cohort intersections

## correlations

In [7]:
df[[c for c in df.columns if 'count' in c] + ['recurring']].corr()

,don_form_count,donations_count,fb_count,givi_count,kiosk_count,mobile_count,mobilevt_count,p2p_count,purchases_count,registrations_count,sms_count,smspledge_count,vt_count,recurring
don_form_count,1.000000,0.140133,0.091668,0.002908,0.091775,0.029267,1.000000,-0.025392,-0.004159,-0.016467,0.091468,NaN,NaN,0.306601
donations_count,0.140133,1.000000,0.998257,0.005704,0.997468,0.068058,0.140133,0.117390,0.000905,0.029671,0.996916,NaN,NaN,0.976428
fb_count,0.091668,0.998257,1.000000,-0.000021,0.998193,0.054864,0.091668,0.117368,0.001125,0.030402,0.998067,NaN,NaN,0.966757
givi_count,0.002908,0.005704,-0.000021,1.000000,0.004781,0.013008,0.002908,-0.003108,0.000549,-0.001030,0.000575,NaN,NaN,0.000560
kiosk_count,0.091775,0.997468,0.998193,0.004781,1.000000,0.056763,0.091775,0.116615,0.001055,0.030050,0.996773,NaN,NaN,0.965581
mobile_count,0.029267,0.068058,0.054864,0.013008,0.056763,1.000000,0.029267,-0.030574,-0.003918,-0.016038,0.056558,NaN,NaN,0.059789
mobilevt_count,1.000000,0.140133,0.091668,0.002908,0.091775,0.029267,1.000000,-0.025392,-0.004159,-0.016467,0.091468,NaN,NaN,0.306601
p2p_count,-0.025392,0.117390,0.117368,-0.003108,0.116615,-0.030574,-0.025392,1.000000,0.084632,0.514769,0.116451,NaN,NaN,0.113401
purchases_count,-0.004159,0.000905,0.001125,0.000549,0.001055,-0.003918,-0.004159,0.084632,1.000000,0.066788,0.001061,NaN,NaN,0.000867
registrations_count,-0.016467,0.029671,0.030402,-0.001030,0.030050,-0.016038,-0.016467,0.514769,0.066788,1.000000,0.029973,NaN,NaN,0.029125


In [8]:
df[[c for c in df.columns if 'amount' in c] + ['recurring']].corr()

,avg_amount,don_form_amount,fb_amount,givi_amount,kiosk_amount,mobile_amount,mobilevt_amount,p2p_amount,recurring_amount,recurring_total_amount,sms_amount,total_amount,vt_amount,recurring
avg_amount,1.000000e+00,0.999906,-7.609418e-07,0.000040,0.000002,0.006234,0.000001,0.001283,0.999935,0.999201,0.000019,0.985579,0.000008,0.000117
don_form_amount,9.999058e-01,1.000000,5.649022e-04,0.000034,0.000564,0.003052,0.000560,0.000006,0.999933,0.999243,0.000556,0.985734,0.000563,0.001307
fb_amount,-7.609418e-07,0.000565,1.000000e+00,-0.000020,0.988552,0.004521,0.998598,0.000067,0.003732,0.037857,0.985427,0.168375,0.998638,0.965744
givi_amount,3.988768e-05,0.000034,-1.967558e-05,1.000000,0.016785,0.003267,-0.000014,0.000287,0.000028,0.000008,0.000733,0.001111,0.000037,0.000390
kiosk_amount,2.134973e-06,0.000564,9.885525e-01,0.016785,1.000000,0.005498,0.989664,-0.000304,0.003700,0.037548,0.977122,0.167157,0.989701,0.957341
mobile_amount,6.234348e-03,0.003052,4.521362e-03,0.003267,0.005498,1.000000,0.004312,-0.000590,0.008988,0.009228,0.005136,0.012945,0.004322,0.005828
mobilevt_amount,1.407419e-06,0.000560,9.985980e-01,-0.000014,0.989664,0.004312,1.000000,0.000151,0.003737,0.037895,0.986549,0.168558,0.999779,0.966814
p2p_amount,1.282883e-03,0.000006,6.706289e-05,0.000287,-0.000304,-0.000590,0.000151,1.000000,0.000712,-0.000132,0.000063,0.001579,0.000189,-0.000349
recurring_amount,9.999351e-01,0.999933,3.731861e-03,0.000028,0.003700,0.008988,0.003737,0.000712,1.000000,0.999402,0.003697,0.986258,0.003741,0.003813
recurring_total_amount,9.992010e-01,0.999243,3.785735e-02,0.000008,0.037548,0.009228,0.037895,-0.000132,0.999402,1.000000,0.037394,0.991319,0.037908,0.037152


In [9]:
gt_zero = df[[c for c in df.columns if 'count' in c] + ['recurring']]>0

In [10]:
print("Correlation between counts > 0")
gt_zero_corr = gt_zero.corr()
gt_zero_corr

Correlation between counts > 0


,don_form_count,donations_count,fb_count,givi_count,kiosk_count,mobile_count,mobilevt_count,p2p_count,purchases_count,registrations_count,sms_count,smspledge_count,vt_count,recurring
don_form_count,1.000000,0.072563,-0.002384,-0.015876,-0.055457,-0.270172,1.000000,-0.679127,-0.071940,-0.351931,-0.137196,NaN,NaN,NaN
donations_count,0.072563,1.000000,0.007797,-0.098229,0.007581,-0.065148,0.072563,0.010004,-0.109799,-0.295913,0.067795,NaN,NaN,NaN
fb_count,-0.002384,0.007797,1.000000,-0.001381,0.005928,0.000087,-0.002384,-0.008144,0.001415,-0.004121,0.000258,NaN,NaN,NaN
givi_count,-0.015876,-0.098229,-0.001381,1.000000,0.023146,-0.015615,-0.015876,-0.030794,0.039185,-0.009044,-0.002309,NaN,NaN,NaN
kiosk_count,-0.055457,0.007581,0.005928,0.023146,1.000000,0.016417,-0.055457,-0.042940,-0.005904,-0.022257,0.014430,NaN,NaN,NaN
mobile_count,-0.270172,-0.065148,0.000087,-0.015615,0.016417,1.000000,-0.270172,-0.197038,-0.026421,-0.098923,-0.022866,NaN,NaN,NaN
mobilevt_count,1.000000,0.072563,-0.002384,-0.015876,-0.055457,-0.270172,1.000000,-0.679127,-0.071940,-0.351931,-0.137196,NaN,NaN,NaN
p2p_count,-0.679127,0.010004,-0.008144,-0.030794,-0.042940,-0.197038,-0.679127,1.000000,0.134789,0.477421,-0.075098,NaN,NaN,NaN
purchases_count,-0.071940,-0.109799,0.001415,0.039185,-0.005904,-0.026421,-0.071940,0.134789,1.000000,0.102682,-0.008390,NaN,NaN,NaN
registrations_count,-0.351931,-0.295913,-0.004121,-0.009044,-0.022257,-0.098923,-0.351931,0.477421,0.102682,1.000000,-0.036728,NaN,NaN,NaN


## overlap counts & percentages

In [11]:
count_cols = gt_zero.columns
count_maps = pd.DataFrame(columns=count_cols, index=count_cols)
count_maps.fillna(0, inplace=True)
for col_1 in count_cols:
    for col_2 in count_cols:
        count_maps.at[col_1, col_2] = len(gt_zero[gt_zero[col_1]&gt_zero[col_2]])

In [12]:
count_maps

,don_form_count,donations_count,fb_count,givi_count,kiosk_count,mobile_count,mobilevt_count,p2p_count,purchases_count,registrations_count,sms_count,smspledge_count,vt_count,recurring
don_form_count,616713,521258,193,2790,1778,29627,616713,27358,2589,9165,3388,0,0,616713
donations_count,521258,823406,342,1727,5465,87821,521258,237548,3932,46446,21001,0,0,823406
fb_count,193,342,348,0,11,41,193,31,6,11,8,0,0,348
givi_count,2790,1727,0,5445,170,265,2790,535,337,331,90,0,0,5445
kiosk_count,1778,5465,11,170,6358,1158,1778,278,16,93,298,0,0,6358
mobile_count,29627,87821,41,265,1158,116329,29627,4757,304,1894,1392,0,0,116329
mobilevt_count,616713,521258,193,2790,1778,29627,616713,27358,2589,9165,3388,0,0,616713
p2p_count,27358,237548,31,535,278,4757,27358,286400,8797,91633,1147,0,0,286400
purchases_count,2589,3932,6,337,16,304,2589,8797,9779,3938,87,0,0,9779
registrations_count,9165,46446,11,331,93,1894,9165,91633,3938,96957,478,0,0,96957


In [13]:
(count_maps / len(gt_zero)) * 100.

,don_form_count,donations_count,fb_count,givi_count,kiosk_count,mobile_count,mobilevt_count,p2p_count,purchases_count,registrations_count,sms_count,smspledge_count,vt_count,recurring
don_form_count,61.6713,52.1258,0.0193,0.2790,0.1778,2.9627,61.6713,2.7358,0.2589,0.9165,0.3388,0.0,0.0,61.6713
donations_count,52.1258,82.3406,0.0342,0.1727,0.5465,8.7821,52.1258,23.7548,0.3932,4.6446,2.1001,0.0,0.0,82.3406
fb_count,0.0193,0.0342,0.0348,0.0000,0.0011,0.0041,0.0193,0.0031,0.0006,0.0011,0.0008,0.0,0.0,0.0348
givi_count,0.2790,0.1727,0.0000,0.5445,0.0170,0.0265,0.2790,0.0535,0.0337,0.0331,0.0090,0.0,0.0,0.5445
kiosk_count,0.1778,0.5465,0.0011,0.0170,0.6358,0.1158,0.1778,0.0278,0.0016,0.0093,0.0298,0.0,0.0,0.6358
mobile_count,2.9627,8.7821,0.0041,0.0265,0.1158,11.6329,2.9627,0.4757,0.0304,0.1894,0.1392,0.0,0.0,11.6329
mobilevt_count,61.6713,52.1258,0.0193,0.2790,0.1778,2.9627,61.6713,2.7358,0.2589,0.9165,0.3388,0.0,0.0,61.6713
p2p_count,2.7358,23.7548,0.0031,0.0535,0.0278,0.4757,2.7358,28.6400,0.8797,9.1633,0.1147,0.0,0.0,28.6400
purchases_count,0.2589,0.3932,0.0006,0.0337,0.0016,0.0304,0.2589,0.8797,0.9779,0.3938,0.0087,0.0,0.0,0.9779
registrations_count,0.9165,4.6446,0.0011,0.0331,0.0093,0.1894,0.9165,9.1633,0.3938,9.6957,0.0478,0.0,0.0,9.6957


# escalating engagement metrics

In [15]:
onetime_msk = df['donations_count']!=0
events_msk = df['events_tickets']!=0
recurring_msk = df['recurring_total_transactions']!=0
p2p_msk = df['p2p_count']!=0
all_masks = {'onetime': onetime_msk, 'events': events_msk, 'recurring': recurring_msk, 'p2p': p2p_msk}
for mask in all_masks:
    _this_group = df[all_masks[mask]]
    _this_perc = float(len(_this_group)) / float(len(df))
    print("{} ({:.2f}%):".format(mask, _this_perc * 100.))
    for second_mask in all_masks:
        if second_mask != mask:
            _other_group = df[all_masks[second_mask]]
            len_overlap = len(_this_group[_this_group['donor'].isin(_other_group['donor'])])
            overlap_perc = float(len_overlap) / float(len(_this_group))
            print("+ {}: {:.2f}%".format(second_mask, overlap_perc * 100.))
    print()

onetime (82.34%):
+ events: 4.41%
+ recurring: 10.27%
+ p2p: 28.85%

events (15.31%):
+ onetime: 23.73%
+ recurring: 3.44%
+ p2p: 6.98%

recurring (8.89%):
+ onetime: 95.17%
+ events: 5.93%
+ p2p: 4.19%

p2p (28.64%):
+ onetime: 82.94%
+ events: 3.73%
+ recurring: 1.30%



In [33]:
onetime_msk = df['donations_count']!=0
events_msk = df['events_tickets']!=0
recurring_msk = df['recurring_total_transactions']!=0
p2p_msk = df['p2p_count']!=0

df_onetime_p2p = df[onetime_msk | p2p_msk]
perc_onetime_p2p = (float(len(df_onetime_p2p)) / float(len(df))) * 100.
perc_plus_p2p = (float(len(df[onetime_msk | p2p_msk | events_msk])) / float(len(df))) * 100.
perc_plus_p2p_rec = (float(len(df[onetime_msk | p2p_msk | events_msk | recurring_msk])) / float(len(df))) * 100.
print("Onetime + p2p ({:.2f}%):".format(perc_onetime_p2p))
print("+ events: {:.2f}%".format(perc_plus_p2p))
print("+ recurring: {:.2f}%".format(perc_plus_p2p_rec))

Onetime + p2p (87.23%):
+ events: 98.74%
+ recurring: 99.12%


In [17]:
def print_trans_stats(df, group, perc):
    print("Total amount:")
    print("\tmean: ${:.2f}".format(df['total_amount'].mean()))
    print("\tmedian: ${:.2f}".format(df['total_amount'].median()))
    print("Transactions count:")
    print("\tmean: {:.2f}".format(df['transactions'].mean()))
    print("\tmedian: {:.2f}".format(df['transactions'].median()))
    print("Average amount:")
    print("\tmean: ${:.2f}".format(df['avg_amount'].mean()))
    print("\tmedian: ${:.2f}".format(df['avg_amount'].median()))
    
    return {
        'group': group,
        'percentage': perc,
        'total amount mean': df['total_amount'].mean(),
        'total amount median': df['total_amount'].median(),
        'trans count mean': df['transactions'].mean(),
        'trans count median': df['transactions'].median(),
        'avg amount mean': df['avg_amount'].mean(),
        'avg amount median': df['avg_amount'].median()
    }

In [18]:
data = []

In [19]:
print("all donors: {:,} (100%)".format(len(df)))
data.append(print_trans_stats(df, 'all', 100))

all donors: 1,000,000 (100%)
Total amount:
	mean: $651.96
	median: $84.20
Transactions count:
	mean: 3.80
	median: 1.00
Average amount:
	mean: $238.72
	median: $50.00


In [20]:
# one time only
# exclude recurring, p2p/registrations
ex_cols = ['p2p_amount', 'p2p_count',
           'recurring_amount', 'recurring_total_amount',
           'recurring_total_transactions', 'registrations_amt',
           'registrations_count', 'events_tickets']
onetime_df = None
for col in ex_cols:
    if onetime_df is None:
        onetime_df = df[df[col]==0]
    else:
        onetime_df = onetime_df[onetime_df[col]==0]

In [21]:
len_onetime = len(onetime_df)
perc_onetime = (len_onetime / len(df)) * 100.
print("one time only: {:,} ({:,.0f}%)".format(len_onetime, perc_onetime))
data.append(print_trans_stats(onetime_df, 'one time only', perc_onetime))

one time only: 301 (0%)
Total amount:
	mean: $125.43
	median: $0.00
Transactions count:
	mean: 1.44
	median: 1.00
Average amount:
	mean: $41.53
	median: $0.00


In [22]:
# only events
ex_cols = ['donations_count', 'fb_amount', 'fb_count',
           'givi_amount', 'givi_count',
           'recurring_total_transactions', 'recurring_total_amount',
           'sms_amount', 'sms_count', 'smspledge_count',
           'vt_amount', 'vt_count']
events_df = None
for col in ex_cols:
    if events_df is None:
        events_df = df[df[col]==0]
    else:
        events_df = events_df[events_df[col]==0]

In [23]:
len_events = len(events_df)
perc_events = (len_events / len(df)) * 100.
print("events only: {:,} ({:,.0f}%)".format(len_events, perc_events))
data.append(print_trans_stats(events_df, 'events only', perc_events))

events only: 164,989 (16%)
Total amount:
	mean: $104.20
	median: $10.00
Transactions count:
	mean: 1.51
	median: 1.00
Average amount:
	mean: $65.11
	median: $10.00


In [24]:
# one time & events
ex_cols = ['recurring_total_transactions', 'recurring_total_amount',
          'smspledge_count']
onetime_events_df = None
for col in ex_cols:
    if onetime_events_df is None:
        onetime_events_df = df[df[col]==0]
    else:
        onetime_events_df = onetime_events_df[onetime_events_df[col]==0]

In [25]:
len_events_onetime = len(onetime_events_df)
perc_events_onetime = (len_events_onetime / len(df)) * 100.
print("one time & events only: {:,} ({:,.0f}%)".format(len_events_onetime, perc_events_onetime))
data.append(print_trans_stats(onetime_events_df, 'one time & events', perc_events_onetime))

one time & events only: 911,120 (91%)
Total amount:
	mean: $324.54
	median: $62.00
Transactions count:
	mean: 2.01
	median: 1.00
Average amount:
	mean: $143.66
	median: $50.00


In [26]:
# recurring only
ex_cols = ['givi_amount', 'givi_count',
           'registrations_amt', 'registrations_count', 'events_tickets']
rec_df = df[df['recurring_total_transactions']!=0]
for col in ex_cols:
    if rec_df is None:
        rec_df = df[df[col]==0]
    else:
        rec_df = rec_df[rec_df[col]==0]

In [27]:
rec_df = df[df['recurring_total_transactions']!=0]
len_rec = len(rec_df)
perc_rec = (len_rec / len(df)) * 100.
print("recurring only: {:,} ({:,.0f}%)".format(len_rec, perc_rec))
data.append(print_trans_stats(rec_df, 'recurring only', perc_rec))

recurring only: 88,880 (9%)
Total amount:
	mean: $4008.41
	median: $420.00
Transactions count:
	mean: 22.21
	median: 12.00
Average amount:
	mean: $1213.13
	median: $30.00


In [28]:
# one time or recurring
ex_cols = ['p2p_amount', 'p2p_count', 'registrations_amt',
           'registrations_count', 'events_tickets']
recurring_df = None
for col in ex_cols:
    if recurring_df is None:
        recurring_df = df[df[col]==0]
    else:
        recurring_df = recurring_df[recurring_df[col]==0]

In [29]:
len_recurring = len(recurring_df)
perc_recurring = (len_recurring / len(df)) * 100.
print("one time or recurring only: {:,} ({:,.0f}%)".format(len_recurring, perc_recurring))
data.append(print_trans_stats(recurring_df, 'one time & recurring', perc_recurring))

one time or recurring only: 565,968 (57%)
Total amount:
	mean: $865.05
	median: $100.00
Transactions count:
	mean: 4.37
	median: 1.00
Average amount:
	mean: $344.94
	median: $52.00


In [30]:
# p2p only
ex_cols = ['don_form_amount', 'don_form_count',
           'donations_count', 'fb_amount', 'fb_count',
           'givi_amount', 'givi_count', 'kiosk_amount',
           'kiosk_count', 'mobile_amount', 'mobile_count',
           'mobilevt_amount', 'mobilevt_count',
           'sms_amount', 'sms_count', 'smspledge_count',
           'vt_amount', 'vt_count', 'registrations_amt',
           'registrations_count', 'smspledge_count',
           'recurring_amount', 'recurring_total_amount', 'events_tickets']
p2p_df = None
for col in ex_cols:
    if p2p_df is None:
        p2p_df = df[df[col]==0]
    else:
        p2p_df = p2p_df[p2p_df[col]==0]

In [31]:
len_p2p = len(p2p_df)
perc_p2p = (len_p2p / len(df)) * 100.
print("p2p only: {:,} ({:,.0f}%)".format(len_p2p, perc_p2p))
data.append(print_trans_stats(p2p_df, 'P2P only', perc_p2p))

p2p only: 12 (0%)
Total amount:
	mean: $41.67
	median: $0.00
Transactions count:
	mean: 1.83
	median: 1.00
Average amount:
	mean: $20.83
	median: $0.00


In [32]:
cols = ['group', 'percentage', 'total amount mean', 'total amount median', 'trans count mean', 'trans count median']
grp_isolation = pd.DataFrame(data)
grp_isolation['percentage'] = grp_isolation['percentage'].astype(float)
grp_isolation[cols].sort_values('percentage')

,group,percentage,total amount mean,total amount median,trans count mean,trans count median
6,P2P only,0.0012,41.666667,0.0,1.833333,1.0
1,one time only,0.0301,125.432691,0.0,1.435216,1.0
4,recurring only,8.8880,4008.413172,420.0,22.206402,12.0
2,events only,16.4989,104.204926,10.0,1.512834,1.0
5,one time & recurring,56.5968,865.051668,100.0,4.366224,1.0
3,one time & events,91.1120,324.538790,62.0,2.007314,1.0
0,all,100.0000,651.961545,84.2,3.802609,1.0


# examining escalation

The path of escalation by overlap percentages appears to be __one time > P2P > events > recurring__. Here we want to examine what attributes we can that strongly correlate between overlapping groups that might indicate common triggers to escalation.

In [71]:
onetime_msk = df['donations_count']!=0
events_msk = df['events_tickets']!=0
recurring_msk = df['recurring_total_transactions']!=0
p2p_msk = df['p2p_count']!=0
all_masks = {'onetime': onetime_msk, 'p2p': p2p_msk, 'events': events_msk, 'recurring': recurring_msk}

df['first_transaction'] = pd.to_datetime(df['first_transaction'])
df['last_transaction'] = pd.to_datetime(df['last_transaction'])
df['donation_duration'] = (df['last_transaction'] - df['first_transaction']).days

for msk in all_masks:
    mn_dt = df[all_masks[msk]]
    print("{}:".format(msk))
    print("|\tfirst trans: {:%Y-%m-%d}".format(mn_dt['first_transaction'].mean()))
    print("|\tlast trans: {:%Y-%m-%d}".format(mn_dt['last_transaction'].mean()))
    print("|\ttrans duration: mean {} days, median {} days".format(mn_dt['donation_duration'].mean().days, mn_dt['donation_duration'].median().days))
    print("|\ttrans count: {:.2f} mean, {:.2f} median".format(mn_dt['transactions'].mean(), mn_dt['transactions'].median()))
    print("|\ttotal amount: ${:,.2f} mean, ${:,.2f} median".format(mn_dt['total_amount'].mean(), mn_dt['total_amount'].median()))
    print("|\ttrans value: ${:,.2f} mean, ${:,.2f} median".format((mn_dt['total_amount'] / mn_dt['transactions']).mean(), (mn_dt['total_amount'] / mn_dt['transactions']).median()))
    print()

onetime:
|	first trans: 2020-01-20
|	last trans: 2020-08-07
|	trans duration: mean 199 days, median 0 days
|	trans count: 4.19 mean, 1.00 median
|	total amount: $761.26 mean, $100.00 median
|	trans value: $274.60 mean, $51.97 median

p2p:
|	first trans: 2020-02-06
|	last trans: 2020-07-04
|	trans duration: mean 148 days, median 0 days
|	trans count: 3.07 mean, 1.00 median
|	total amount: $339.63 mean, $52.00 median
|	trans value: $94.79 mean, $50.00 median

events:
|	first trans: 2019-12-13
|	last trans: 2020-07-27
|	trans duration: mean 226 days, median 0 days
|	trans count: 5.51 mean, 1.00 median
|	total amount: $729.94 mean, $50.00 median
|	trans value: $117.48 mean, $30.00 median

recurring:
|	first trans: 2019-04-06
|	last trans: 2020-10-23
|	trans duration: mean 565 days, median 418 days
|	trans count: 22.21 mean, 12.00 median
|	total amount: $4,008.41 mean, $420.00 median
|	trans value: $1,213.13 mean, $30.00 median



In [111]:
df_ot = df[onetime_msk]
df_one_day = df_ot[df_ot['donation_duration'].dt.days==0]
len_one_day_p2p = len(df_one_day[df_one_day['donor'].isin(df[p2p_msk]['donor'].tolist())])
len_one_day_events = len(df_one_day[df_one_day['donor'].isin(df[events_msk]['donor'].tolist())])
len_one_day_rec = len(df_one_day[df_one_day['donor'].isin(df[recurring_msk]['donor'].tolist())])

print("Gave once and never returned: {:.0f}%".format((float(len(df_one_day)) / float(len(df_ot))) * 100.))
print("|\toverlap w/ p2p: {:.2f}%".format((float(len_one_day_p2p) / float(len(df_one_day))) * 100.))
print("|\toverlap w/ events: {:.2f}%".format((float(len_one_day_events) / float(len(df_one_day))) * 100.))
print("|\toverlap w/ recurring: {:.2f}%".format((float(len_one_day_rec) / float(len(df_one_day))) * 100.))
print("|\tmean amount per transaction: ${:.2f}".format((df_one_day['total_amount'] / df_one_day['transactions']).mean()))
print("|\tmedian amount per transaction: ${:.2f}".format((df_one_day['total_amount'] / df_one_day['transactions']).median()))
print("|\tmean total amount: ${:,.2f}".format(df_one_day['total_amount'].mean()))
print()

df_gt_one_day = df_ot[df_ot['donation_duration'].dt.days>0]
perc_gt_one_day = (float(len(df_gt_one_day)) / float(len(df_ot))) * 100.
len_one_day_p2p = len(df_gt_one_day[df_gt_one_day['donor'].isin(df[p2p_msk]['donor'].tolist())])
len_one_day_events = len(df_gt_one_day[df_gt_one_day['donor'].isin(df[events_msk]['donor'].tolist())])
len_one_day_rec = len(df_gt_one_day[df_gt_one_day['donor'].isin(df[recurring_msk]['donor'].tolist())])

print("Gave repeatedly: {:.0f}%".format(perc_gt_one_day))
print("|\toverlap w/ p2p: {:.2f}%".format((float(len_one_day_p2p) / float(len(df_gt_one_day))) * 100.))
print("|\toverlap w/ events: {:.2f}%".format((float(len_one_day_events) / float(len(df_gt_one_day))) * 100.))
print("|\toverlap w/ recurring: {:.2f}%".format((float(len_one_day_rec) / float(len(df_gt_one_day))) * 100.))
print("|\tmean amount per transaction: ${:.2f}".format((df_gt_one_day['total_amount'] / df_gt_one_day['transactions']).mean()))
print("|\tmedian amount per transaction: ${:.2f}".format((df_gt_one_day['total_amount'] / df_gt_one_day['transactions']).median()))
print("|\tmean total amount: ${:,.2f}".format(df_gt_one_day['total_amount'].mean()))
print()

Gave once and never returned: 63%
|	overlap w/ p2p: 31.41%
|	overlap w/ events: 0.28%
|	overlap w/ recurring: 1.01%
|	mean amount per transaction: $144.71
|	median amount per transaction: $50.40
|	mean total amount: $149.54

Gave repeatedly: 37%
|	overlap w/ p2p: 24.54%
|	overlap w/ events: 11.39%
|	overlap w/ recurring: 25.90%
|	mean amount per transaction: $493.65
|	median amount per transaction: $61.25
|	mean total amount: $1,792.82



In [136]:
print("| Days engaged | P2P overlap | Events overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |")
print("|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|")

engagement_days_ranges = [(0), (1, 10), (10, 30), (30, 60), (60, 90), (90)]
for i in engagement_days_ranges:
    try:
        _df = df_ot[(df_ot['donation_duration'].dt.days>=i[0])&(df_ot['donation_duration'].dt.days<i[1])].copy()
        _df = _df[_df['total_amount']!=199999999.98]
    except:
        if i == 0:
            _df = df_ot[df_ot['donation_duration'].dt.days==i].copy()
        else:
            _df = df_ot[df_ot['donation_duration'].dt.days>=i].copy()
        
    _df['avg_trans'] = _df['total_amount'] / _df['transactions']
    len_this_p2p = len(_df[_df['donor'].isin(df[p2p_msk]['donor'].tolist())])
    len_this_events = len(_df[_df['donor'].isin(df[events_msk]['donor'].tolist())])
    len_this_rec = len(_df[_df['donor'].isin(df[recurring_msk]['donor'].tolist())])
        
    overlap_p2p = (float(len_this_p2p) / float(len(_df))) * 100.
    overlap_events = (float(len_this_events) / float(len(_df))) * 100.
    overlap_recurring = (float(len_this_rec) / float(len(_df))) * 100.
    trans_mean = _df['avg_trans'].mean()
    trans_median = _df['avg_trans'].median()
    total_mean = _df['total_amount'].mean()
    total_median = _df['total_amount'].median()
    print("| {} | {:.2f}% | {:.2f}% | {:.2f}% | \${:,.2f} | \${:,.2f} | \${:,.2f} | \${:,.2f} |".format(i, overlap_p2p, overlap_events, overlap_recurring, trans_mean, trans_median, total_mean, total_median))

| Days engaged | P2P ovelap | Events overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 31.41% | 0.28% | 1.01% | \$144.71 | \$50.40 | \$149.54 | \$51.50 |
| (1, 10) | 53.84% | 10.24% | 3.47% | \$138.59 | \$50.00 | \$314.99 | \$100.00 |
| (10, 30) | 37.77% | 8.14% | 3.64% | \$143.76 | \$52.50 | \$479.31 | \$125.00 |
| (30, 60) | 21.62% | 6.90% | 24.63% | \$129.32 | \$50.00 | \$342.46 | \$125.00 |
| (60, 90) | 17.92% | 8.38% | 29.38% | \$141.80 | \$50.74 | \$388.50 | \$150.00 |
| 90 | 23.49% | 11.89% | 27.46% | \$172.05 | \$62.88 | \$1,258.88 | \$307.50 |


| Days engaged | P2P ovelap | Events overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 31.41% | 0.28% | 1.01% | \$144.71 | \$50.40 | \$149.54 | \$51.50 |
| (1, 10) | 53.84% | 10.24% | 3.47% | \$138.59 | \$50.00 | \$314.99 | \$100.00 |
| (10, 30) | 37.77% | 8.14% | 3.64% | \$143.76 | \$52.50 | \$479.31 | \$125.00 |
| (30, 60) | 21.62% | 6.90% | 24.63% | \$129.32 | \$50.00 | \$342.46 | \$125.00 |
| (60, 90) | 17.92% | 8.38% | 29.38% | \$141.80 | \$50.74 | \$388.50 | \$150.00 |
| 90+ | 23.49% | 11.89% | 27.46% | \$172.05 | \$62.88 | \$1,258.88 | \$307.50 |

In [139]:
df_p2p = df[p2p_msk]

print("| Days engaged | Onetime overlap | Events overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |")
print("|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|")

engagement_days_ranges = [(0), (1, 10), (10, 30), (30, 60), (60, 90), (90)]
for i in engagement_days_ranges:
    try:
        _df = df_p2p[(df_p2p['donation_duration'].dt.days>=i[0])&(df_p2p['donation_duration'].dt.days<i[1])].copy()
        _df = _df[_df['total_amount']!=199999999.98]
    except:
        if i == 0:
            _df = df_p2p[df_p2p['donation_duration'].dt.days==i].copy()
        else:
            _df = df_p2p[df_p2p['donation_duration'].dt.days>=i].copy()
        
    _df['avg_trans'] = _df['total_amount'] / _df['transactions']
    len_this_onetime = len(_df[_df['donor'].isin(df[onetime_msk]['donor'].tolist())])
    len_this_events = len(_df[_df['donor'].isin(df[events_msk]['donor'].tolist())])
    len_this_rec = len(_df[_df['donor'].isin(df[recurring_msk]['donor'].tolist())])
        
    overlap_onetime = (float(len_this_onetime) / float(len(_df))) * 100.
    overlap_events = (float(len_this_events) / float(len(_df))) * 100.
    overlap_recurring = (float(len_this_rec) / float(len(_df))) * 100.
    trans_mean = _df['avg_trans'].mean()
    trans_median = _df['avg_trans'].median()
    total_mean = _df['total_amount'].mean()
    total_median = _df['total_amount'].median()
    print("| {} | {:.2f}% | {:.2f}% | {:.2f}% | \${:,.2f} | \${:,.2f} | \${:,.2f} | \${:,.2f} |".format(i, overlap_onetime, overlap_events, overlap_recurring, trans_mean, trans_median, total_mean, total_median))

| Days engaged | Onetime overlap | Events overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 80.44% | 0.04% | 0.07% | \$81.57 | \$40.00 | \$84.56 | \$41.98 |
| (1, 10) | 93.41% | 3.75% | 0.41% | \$84.53 | \$35.00 | \$195.64 | \$80.00 |
| (10, 30) | 94.46% | 5.41% | 0.52% | \$93.85 | \$42.50 | \$225.63 | \$100.00 |
| (30, 60) | 94.45% | 9.30% | 2.10% | \$116.59 | \$50.00 | \$303.30 | \$115.00 |
| (60, 90) | 94.28% | 13.99% | 5.44% | \$119.60 | \$52.47 | \$312.27 | \$130.00 |
| 90 | 87.91% | 13.65% | 4.77% | \$131.64 | \$52.60 | \$1,085.52 | \$171.14 |


| Days engaged | Onetime overlap | Events overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 80.44% | 0.04% | 0.07% | \$81.57 | \$40.00 | \$84.56 | \$41.98 |
| (1, 10) | 93.41% | 3.75% | 0.41% | \$84.53 | \$35.00 | \$195.64 | \$80.00 |
| (10, 30) | 94.46% | 5.41% | 0.52% | \$93.85 | \$42.50 | \$225.63 | \$100.00 |
| (30, 60) | 94.45% | 9.30% | 2.10% | \$116.59 | \$50.00 | \$303.30 | \$115.00 |
| (60, 90) | 94.28% | 13.99% | 5.44% | \$119.60 | \$52.47 | \$312.27 | \$130.00 |
| 90+ | 87.91% | 13.65% | 4.77% | \$131.64 | \$52.60 | \$1,085.52 | \$171.14 |

In [140]:
df_events = df[events_msk]

print("| Days engaged | Onetime overlap | P2P overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |")
print("|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|")

engagement_days_ranges = [(0), (1, 10), (10, 30), (30, 60), (60, 90), (90)]
for i in engagement_days_ranges:
    try:
        _df = df_events[(df_events['donation_duration'].dt.days>=i[0])&(df_events['donation_duration'].dt.days<i[1])].copy()
        _df = _df[_df['total_amount']!=199999999.98]
    except:
        if i == 0:
            _df = df_events[df_events['donation_duration'].dt.days==i].copy()
        else:
            _df = df_events[df_events['donation_duration'].dt.days>=i].copy()
        
    _df['avg_trans'] = _df['total_amount'] / _df['transactions']
    len_this_onetime = len(_df[_df['donor'].isin(df[onetime_msk]['donor'].tolist())])
    len_this_p2p = len(_df[_df['donor'].isin(df[p2p_msk]['donor'].tolist())])
    len_this_rec = len(_df[_df['donor'].isin(df[recurring_msk]['donor'].tolist())])
        
    overlap_onetime = (float(len_this_onetime) / float(len(_df))) * 100.
    overlap_p2p = (float(len_this_p2p) / float(len(_df))) * 100.
    overlap_recurring = (float(len_this_rec) / float(len(_df))) * 100.
    trans_mean = _df['avg_trans'].mean()
    trans_median = _df['avg_trans'].median()
    total_mean = _df['total_amount'].mean()
    total_median = _df['total_amount'].median()
    print("| {} | {:.2f}% | {:.2f}% | {:.2f}% | \${:,.2f} | \${:,.2f} | \${:,.2f} | \${:,.2f} |".format(i, overlap_onetime, overlap_p2p, overlap_recurring, trans_mean, trans_median, total_mean, total_median))

| Days engaged | Onetime overlap | P2P overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 1.60% | 0.10% | 0.02% | \$77.97 | \$13.65 | \$80.81 | \$15.00 |
| (1, 10) | 18.30% | 3.86% | 0.59% | \$114.23 | \$40.00 | \$247.56 | \$88.40 |
| (10, 30) | 24.27% | 6.45% | 0.33% | \$113.70 | \$37.50 | \$254.03 | \$80.00 |
| (30, 60) | 34.34% | 10.60% | 1.50% | \$106.81 | \$32.50 | \$256.41 | \$77.50 |
| (60, 90) | 42.76% | 13.57% | 2.32% | \$115.13 | \$38.22 | \$297.45 | \$98.50 |
| 90 | 60.93% | 18.68% | 9.93% | \$186.51 | \$71.43 | \$1,959.87 | \$290.00 |


| Days engaged | Onetime overlap | P2P overlap | Recurring overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 1.60% | 0.10% | 0.02% | \$77.97 | \$13.65 | \$80.81 | \$15.00 |
| (1, 10) | 18.30% | 3.86% | 0.59% | \$114.23 | \$40.00 | \$247.56 | \$88.40 |
| (10, 30) | 24.27% | 6.45% | 0.33% | \$113.70 | \$37.50 | \$254.03 | \$80.00 |
| (30, 60) | 34.34% | 10.60% | 1.50% | \$106.81 | \$32.50 | \$256.41 | \$77.50 |
| (60, 90) | 42.76% | 13.57% | 2.32% | \$115.13 | \$38.22 | \$297.45 | \$98.50 |
| 90+ | 60.93% | 18.68% | 9.93% | \$186.51 | \$71.43 | \$1,959.87 | \$290.00 |

In [141]:
df_recurring = df[recurring_msk]

print("| Days engaged | Onetime overlap | P2P overlap | Events overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |")
print("|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|")

engagement_days_ranges = [(0), (1, 10), (10, 30), (30, 60), (60, 90), (90)]
for i in engagement_days_ranges:
    try:
        _df = df_recurring[(df_recurring['donation_duration'].dt.days>=i[0])&(df_recurring['donation_duration'].dt.days<i[1])].copy()
        _df = _df[_df['total_amount']!=199999999.98]
    except:
        if i == 0:
            _df = df_recurring[df_recurring['donation_duration'].dt.days==i].copy()
        else:
            _df = df_recurring[df_recurring['donation_duration'].dt.days>=i].copy()
        
    _df['avg_trans'] = _df['total_amount'] / _df['transactions']
    len_this_onetime = len(_df[_df['donor'].isin(df[onetime_msk]['donor'].tolist())])
    len_this_p2p = len(_df[_df['donor'].isin(df[p2p_msk]['donor'].tolist())])
    len_this_events = len(_df[_df['donor'].isin(df[events_msk]['donor'].tolist())])
        
    overlap_onetime = (float(len_this_onetime) / float(len(_df))) * 100.
    overlap_p2p = (float(len_this_p2p) / float(len(_df))) * 100.
    overlap_events = (float(len_this_events) / float(len(_df))) * 100.
    trans_mean = _df['avg_trans'].mean()
    trans_median = _df['avg_trans'].median()
    total_mean = _df['total_amount'].mean()
    total_median = _df['total_amount'].median()
    print("| {} | {:.2f}% | {:.2f}% | {:.2f}% | \${:,.2f} | \${:,.2f} | \${:,.2f} | \${:,.2f} |".format(i, overlap_onetime, overlap_p2p, overlap_events, trans_mean, trans_median, total_mean, total_median))

| Days engaged | Onetime overlap | P2P overlap | Events overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 87.66% | 2.49% | 0.25% | \$261.91 | \$31.20 | \$364.25 | \$64.00 |
| (1, 10) | 90.14% | 6.12% | 8.50% | \$113.88 | \$25.00 | \$342.06 | \$60.90 |
| (10, 30) | 94.21% | 5.35% | 2.90% | \$334.26 | \$26.25 | \$4,829.24 | \$95.00 |
| (30, 60) | 97.67% | 1.90% | 1.20% | \$56.75 | \$25.00 | \$258.57 | \$75.00 |
| (60, 90) | 97.04% | 3.42% | 1.50% | \$49.83 | \$25.00 | \$234.04 | \$100.00 |
| 90 | 95.60% | 4.44% | 6.75% | \$75.69 | \$30.93 | \$1,975.90 | \$525.00 |


| Days engaged | Onetime overlap | P2P overlap | Events overlap | Mean per transaction | Median per transaction | Mean total amount | Median total amount |
|-------------|-------------|-------------|-------------|--------------|--------------|--------------|--------------|
| 0 | 87.66% | 2.49% | 0.25% | \$261.91 | \$31.20 | \$364.25 | \$64.00 |
| (1, 10) | 90.14% | 6.12% | 8.50% | \$113.88 | \$25.00 | \$342.06 | \$60.90 |
| (10, 30) | 94.21% | 5.35% | 2.90% | \$334.26 | \$26.25 | \$4,829.24 | \$95.00 |
| (30, 60) | 97.67% | 1.90% | 1.20% | \$56.75 | \$25.00 | \$258.57 | \$75.00 |
| (60, 90) | 97.04% | 3.42% | 1.50% | \$49.83 | \$25.00 | \$234.04 | \$100.00 |
| 90+ | 95.60% | 4.44% | 6.75% | \$75.69 | \$30.93 | \$1,975.90 | \$525.00 |